In [1]:
import sys
!{sys.executable} -m pip install numpy

In [2]:
import sys
!{sys.executable} -m pip install pandas
import sys
!{sys.executable} -m pip install streamlit
import sys
!{sys.executable} -m pip install requests
import sys
!{sys.executable} -m pip install bz2file


In [3]:
import sys
!{sys.executable} -m pip install scikit-learn


In [4]:
import sys
!{sys.executable} -m pip install nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /home/chandan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/chandan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/chandan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from functools import reduce
 
ps = PorterStemmer()

In [6]:
import numpy as np 
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity


In [8]:
#Reading the given dataset
policy_data=pd.read_csv("recommendation_dataset.csv")

In [9]:
#Removing all entries with ages <0
policy_data=policy_data[(policy_data.age>0)]

In [10]:
#Removing all entries with ages >100
policy_data=policy_data[(policy_data.age<=100)]


In [11]:
# Printing the structure of the given data for better understanding
policy_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114404 entries, 0 to 114470
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   user_id              114404 non-null  int64 
 1   gender               114404 non-null  object
 2   social_category      114404 non-null  object
 3   dob                  114404 non-null  object
 4   age                  114404 non-null  int64 
 5   scheme_id            114404 non-null  int64 
 6   scheme_name          114404 non-null  object
 7   domicile_of_tripura  114404 non-null  object
 8   description          114404 non-null  object
dtypes: int64(3), object(6)
memory usage: 8.7+ MB


In [12]:
#Ensuring that there are no null entries in the data
policy_data.isnull().sum()

user_id                0
gender                 0
social_category        0
dob                    0
age                    0
scheme_id              0
scheme_name            0
domicile_of_tripura    0
description            0
dtype: int64

In [13]:
#Ensuring there are no duplicated entries in the dataset
policy_data.duplicated().sum()

0

In [14]:
#Displaying number of records and fields
records=policy_data.shape[0]
fields=policy_data.shape[1]
print("Records: ",records)
print("Fields: ",fields)


Records:  114404
Fields:  9


In [15]:
# Grouping dataset according to scheme_id
ans=policy_data.groupby(by="scheme_id")

In [16]:
description_matrix = pd.get_dummies(policy_data['description'].str.split(" ").apply(pd.Series).stack())

In [17]:
# Here we are creating a description matrix in which each description is split on the basis of space and then every row shows that the description of that scheme has the given word or not 
# description_matrix = pd.get_dummies(policy_data["description"].apply(word_tokenize).apply(pd.Series).stack())


In [18]:
#Converting the scheme_name column into a list of scheme names in order to compare them with each other
schemes=policy_data['scheme_name'].tolist()
description=policy_data['description'].str.split(" ").tolist()

In [19]:
def create_bow(feature_list):
    bow = {}
    for feature in feature_list:
        bow[feature] = 1
    return bow
bag_of_words=[create_bow(schemes)for schemes in description]
#Here we are creating a list of all the words in our corpus description for the comparison of schemes on the basis of description

In [20]:
#Here we are creating a binary schemes_df dataframe containing the bag of words as its fields 
schemes_df=pd.DataFrame(bag_of_words,index=schemes).fillna(0)
schemes_df= schemes_df[~schemes_df.index.duplicated(keep='first')]
schemes_df

#Removing the fields which can pollute our comparisons
schemes_df=schemes_df.drop("to",axis=1)
schemes_df=schemes_df.drop("for",axis=1)
schemes_df=schemes_df.drop("of",axis=1)
schemes_df=schemes_df.drop("Class",axis=1)
schemes_df=schemes_df.drop("SC",axis=1)
schemes_df=schemes_df.drop("ST",axis=1)
schemes_df=schemes_df.drop("Dr.",axis=1)
schemes_df=schemes_df.drop("B.",axis=1)
schemes_df=schemes_df.drop("R.",axis=1)
schemes_df=schemes_df.drop("DR.",axis=1)
schemes_df=schemes_df.drop("OBC",axis=1)
schemes_df=schemes_df.drop("OR",axis=1)
schemes_df=schemes_df.drop("under",axis=1)
schemes_df=schemes_df.drop("R",axis=1)
schemes_df=schemes_df.drop("and",axis=1)
schemes_df=schemes_df.drop("B",axis=1)
schemes_df=schemes_df.drop("B.R.",axis=1)
schemes_df=schemes_df.drop("at",axis=1)
schemes_df=schemes_df.drop("-",axis=1)
schemes_df=schemes_df.drop("For",axis=1)
schemes_df=schemes_df.drop("_",axis=1)
schemes_df=schemes_df.drop("\r\nfor",axis=1)
schemes_df=schemes_df.drop("Only",axis=1)


In [21]:


#List of all unique schemes that are available
list_of_all_schemes = []
[list_of_all_schemes.append(x) for x in policy_data["scheme_name"] if x not in list_of_all_schemes]




#mean_age dictionary stores the mean age of applicants for that scheme
mean_age={}

#domicile dictionary stores the percentage of people who have the state domicile for the given scheme
domicile={}

#SC dictionary stores the number of people of SC category (out of 1000) who have applied for that scheme
SC={}

#ST dictionary stores the number of people of ST category (out of 1000) who have applied for that scheme
ST={}

#OBC dictionary stores the number of people of OBC category (out of 1000) who have applied for that scheme
OBC={}

#Others dictionary stores the number of people of Others category (out of 1000) who have applied for that scheme
Others={}

#Male dictionary stores the percentage of Male applicants for that scheme
Male={}

#Female dictionary stores the percentage of Female applicants fo rthat scheme
Female={}

#Transgenders dictionary stores the percentage of Transgenders applicants fo rthat scheme
Transgenders={}

descriptions={}
for x in list_of_all_schemes:
    new = policy_data["scheme_name"].isin([x])
    mean_age[x]=(policy_data[new]["age"].mean())
    domicile[x]=((policy_data[new].query('domicile_of_tripura=="Y"').domicile_of_tripura.count())/policy_data[new].shape[0])
    SC[x]=((policy_data[new].query('social_category=="SC"').social_category.count())/policy_data[new].shape[0])*1000
    ST[x]=((policy_data[new].query('social_category=="ST"').social_category.count())/policy_data[new].shape[0])*1000
    OBC[x]=((policy_data[new].query('social_category=="OBC"').social_category.count())/policy_data[new].shape[0])*1000
    Others[x]=((policy_data[new].query('social_category=="Others"').social_category.count())/policy_data[new].shape[0])*1000
    Male[x]=((policy_data[new].query('gender=="M"').gender.count())/policy_data[new].shape[0])*100
    Female[x]=((policy_data[new].query('gender=="F"').gender.count())/policy_data[new].shape[0])*100
    Transgenders[x]=((policy_data[new].query('gender=="T"').gender.count())/policy_data[new].shape[0])*100
    for y in policy_data[new]["description"]:
        descriptions[x]=y
   

#Adding fields to fine tune our recommendation system
    
schemes_df.insert(0,"age",mean_age,True)
schemes_df.insert(0,"domicile_of_tripura",domicile,True)
schemes_df.insert(2,"SC",SC,True)
schemes_df.insert(2,"ST",ST,True)
schemes_df.insert(2,"OBC",OBC,True)
schemes_df.insert(2,"Others",Others,True)
schemes_df.insert(2,"Male",Male,True)
schemes_df.insert(2,"Female",Female,True)
schemes_df.insert(2,"Transgenders",Transgenders,True)

schemes_df= schemes_df[~schemes_df.index.duplicated(keep='first')]

schemes_df


,domicile_of_tripura,age,Transgenders,Female,Male,Others,OBC,ST,SC,Pre-Matric,...,Year,2023/24,One-Time,Support,Economically,Weaker,Meritorious,2023/2024,Lateral,Entry
Pre-Matric Scholarship for ST (VI to VIII),0.797183,14.131612,0.008119,54.658385,45.333496,0.852515,2.192181,986.806317,10.148987,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mukhyamantri Yuba Yogajog Yojana,0.995132,23.349734,0.005761,52.225263,47.766095,269.566470,274.031399,247.760334,208.641797,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pre-Matric Scholarship Class VI to VIII for SC Students,0.936581,14.020560,0.007240,52.240643,47.752118,0.579165,3.764570,5.502063,990.154203,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dr. B. R. Ambedkar Merit Award,0.916254,16.254572,0.000000,59.793198,40.206802,0.854555,13.331055,6.580072,979.234319,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Stipend for ITI Training,1.000000,22.930197,0.000000,34.597876,65.402124,338.391502,238.239757,238.239757,185.128983,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Merit Award for ST,0.918973,16.905179,0.000000,62.167086,37.832914,0.520517,1.214540,995.662358,2.602585,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pre Matic Scholarship for Minorities,0.878898,14.852067,0.145033,67.730239,32.124728,902.102973,50.036258,25.380711,22.480058,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ambedkar Merit Award,0.901970,18.537196,0.000000,58.609502,41.390498,0.926999,988.644264,0.926999,9.501738,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DR. B. R. Ambedkar Merit Award for OBC,0.998443,17.920610,0.000000,60.367372,39.632628,2.801993,987.858032,0.622665,8.717310,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Stipend for Apprenticeship Training for pass-out Degree OR Diploma Candidates of Civil OR Mechanical OR Electrical Engineering,0.988235,25.676471,0.000000,34.117647,65.882353,300.000000,258.823529,170.588235,270.588235,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
#In this step we apply the cosine similarity function to find the similarity of applicants between different applicants
cosine_similarity = cosine_similarity(schemes_df)



In [23]:
import streamlit as st
import requests
import bz2
import _pickle as cPickle

In [24]:
#Similarity_df is the datframe containing the distances between the applicants of different schemes
similarity_df=pd.DataFrame(cosine_similarity,index=schemes_df.index,columns=schemes_df.index)

def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f: 
        cPickle.dump(data, f)
compressed_pickle('df_new', schemes_df.to_dict()) 
compressed_pickle('similarity', similarity_df)

In [25]:
def decompress_pickle(file):
    # data = bz2.BZ2File(file, 'rb')
    with bz2.BZ2File(file, 'rb') as f:
        data = cPickle.load(f)
    # data = cPickle.load(data)
    return data

In [26]:
df_new = decompress_pickle('df_new.pbz2')
similarity = decompress_pickle('similarity.pbz2')

In [29]:
scheme_dataframe = pd.DataFrame(df_new)

#Similarity_df is the datframe containing the distances between the applicants of different schemes
similarity_df=pd.DataFrame(cosine_similarity,index=scheme_dataframe.index,columns=scheme_dataframe.index)

In [30]:
def recommend(policy):
    #scheme index is the location of that scheme in the similarity_df
    scheme_index=similarity_df.index.get_loc(policy)


    #top_5 is storing the top 10 schemes having a similar applicant as the entered scheme
    top_5=similarity_df.iloc[scheme_index].sort_values(ascending=False)[1:6]

    return top_5


In [30]:
#policy is the scheme the user has applied to:
policy=input("Enter a Scheme you applied :")

recommend(policy)

Enter a Scheme you applied : Pre-Matric Scholarship for ST (VI to VIII)


Pre-Matric Scholarship for ST  VI to VIII                                     0.999968
Merit Award for ST                                                            0.999906
Merit Award to the meritorious ST students                                    0.999904
Lakhshay-Chief Minister's Special Scholarship                                 0.999864
One-Time Financial Support for Economically Weaker Meritorious ST students    0.999643
Name: Pre-Matric Scholarship for ST (VI to VIII), dtype: float64